In [18]:
# adding dependencies

import openai
import os
import pandas as pd
import numpy as np
import json
import tiktoken
import psycopg2
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector


In [21]:
# Loading open AI key

from dotenv import load_dotenv

load_dotenv()


openai.api_key =  os.getenv('OPENAI_API_KEY')

In [22]:
# Load your CSV file into a pandas DataFrame
df = pd.read_csv('blog_posts_data.csv')
df.head()

,title,content,url
0,"How to Build a Weather Station With Elixir, Ne...",This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-to-build-a-...
1,CloudQuery on Using PostgreSQL for Cloud Asset...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/cloudquery-on-u...
2,How a Data Scientist Is Building a Time-Series...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-a-data-scie...
3,How Conserv Safeguards History: Building an En...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-conserv-saf...
4,How Messari Uses Data to Open the Cryptoeconom...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-messari-use...


In [23]:
# Helper functions to help us create the embeddings

# Helper func: calculate number of tokens
def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Helper function: calculate length of essay
def get_essay_length(essay):
    word_list = essay.split()
    num_words = len(word_list)
    return num_words

# Helper function: calculate cost of embedding num_tokens
# Assumes we're using the text-embedding-ada-002 model
# See https://openai.com/pricing
def get_embedding_cost(num_tokens):
    return num_tokens/1000*0.0001

# Helper function: calculate total cost of embedding all content in the dataframe
def get_total_embeddings_cost():
    total_tokens = 0
    for i in range(len(df.index)):
        text = df['content'][i]
        token_len = num_tokens_from_string(text)
        total_tokens = total_tokens + token_len
    total_cost = get_embedding_cost(total_tokens)
    return total_cost



In [24]:
# quick check on total token amount for price estimation
total_cost = get_total_embeddings_cost()
print("estimated price to embed this content = $" + str(total_cost))



estimated price to embed this content = $0.0060178


In [15]:

# Create new list with small content chunks to not hit max token limits
# Note: the maximum number of tokens for a single request is 8191
# https://openai.com/docs/api-reference/requests

# list for chunked content and embeddings
new_list = []
# Split up the text into token sizes of around 512 tokens
for i in range(len(df.index)):
    text = df['content'][i]
    token_len = num_tokens_from_string(text)
    if token_len <= 512:
        new_list.append([df['title'][i], df['content'][i], df['url'][i], token_len])
    else:
        # add content to the new list in chunks
        start = 0
        ideal_token_size = 512
        # 1 token ~ 3/4 of a word
        ideal_size = int(ideal_token_size // (4/3))
        end = ideal_size
        #split text by spaces into words
        words = text.split()

        #remove empty spaces
        words = [x for x in words if x != ' ']

        total_words = len(words)
        
        #calculate iterations
        chunks = total_words // ideal_size
        if total_words % ideal_size != 0:
            chunks += 1
        
        new_content = []
        for j in range(chunks):
            if end > total_words:
                end = total_words
            new_content = words[start:end]
            new_content_string = ' '.join(new_content)
            new_content_token_len = num_tokens_from_string(new_content_string)
            if new_content_token_len > 0:
                new_list.append([df['title'][i], new_content_string, df['url'][i], new_content_token_len])
            start += ideal_size
            end += ideal_size



In [16]:
# Helper function: get embeddings for a text
def get_embeddings(text):
   response = openai.Embedding.create(
       model="text-embedding-ada-002",
       input = text.replace("\n"," ")
   )
   embedding = response['data'][0]['embedding']
   return embedding


In [17]:
# Create embeddings for each piece of content
for i in range(len(new_list)):
   text = new_list[i][1]
   embedding = get_embeddings(text)
   new_list[i].append(embedding)

# Create a new dataframe from the list
df_new = pd.DataFrame(new_list, columns=['title', 'content', 'url', 'tokens', 'embeddings'])
df_new.head()



,title,content,url,tokens,embeddings
0,"How to Build a Weather Station With Elixir, Ne...",This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-to-build-a-...,501,"[0.021440856158733368, 0.02200360782444477, -0..."
1,"How to Build a Weather Station With Elixir, Ne...",capture weather and environmental data. In all...,https://www.timescale.com/blog/how-to-build-a-...,512,"[0.01620873250067234, 0.011362895369529724, 0...."
2,"How to Build a Weather Station With Elixir, Ne...",command in their database migration:SELECT cre...,https://www.timescale.com/blog/how-to-build-a-...,374,"[0.022517921403050423, -0.0019158280920237303,..."
3,CloudQuery on Using PostgreSQL for Cloud Asset...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/cloudquery-on-u...,519,"[0.008915113285183907, -0.004873732570558786, ..."
4,CloudQuery on Using PostgreSQL for Cloud Asset...,Architecture with CloudQuery SDK- Writing plug...,https://www.timescale.com/blog/cloudquery-on-u...,511,"[0.020440293475985527, 0.010124118998646736, 0..."


In [ ]:
# Save the dataframe with embeddings as a CSV file
df_new.to_csv('blog_data_and_embeddings.csv', index=False)
